# **Voice of Customer**

Ampika Thanakaew ID: 6210422049

## **Initialize**

In [21]:
!pip install mlxtend
!pip install --upgrade pythainlp
!pip install pyLDAvis
# !pip install sefr_cut

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [22]:
import pandas as pd
import numpy as np
import gensim
import pythainlp
from pythainlp.tokenize import sent_tokenize, word_tokenize
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

# import re
# import seaborn as sns
# import matplotlib.pyplot as plt
# import sefr_cut
# sefr_cut.load_model(engine='tl-deepcut-ws1000')

In [23]:
%cd '/content/drive/MyDrive/BADS 7105 CRM/Voice of Customer/'
%ls

/content/drive/MyDrive/BADS 7105 CRM/Voice of Customer
CustomerReviews.csv


In [24]:
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('/content/drive/MyDrive/BADS 7105 CRM/font/Cordia New Bold.ttf')
mpl.rc('font', family='Cordia New', size=12, weight=200)

**Load Data**

In [57]:
df = pd.read_csv('CustomerReviews.csv')
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


**Tokenize Words**

In [87]:
# Word Correction
word_correction_dict = {
    'โมโม่' : 'momo', 'มากก' : 'มาก', 'กกก' : 'ก', 'แวท' : 'vat', 'พิซเซอเรีย' : 'pizzeria', 'พิซซ่า' : 'pizza', 
    'เก้บ' : 'เก็บ', 'ชูชิ' : 'ซูชิ', 'ซึป' : 'ซุป', 'ชาบูชิ' : 'shabushi', 'อารามณ์' : 'อารมณ์', 'ส่งน' : 'ส่วน', 'สไลต์' : 'สไลด์', 'ประยุคก์' : 'ประยุกต์',
    'ไอติม' : 'icecream', 'พาราไดส์' : 'paradise', 'พาราไดซ์' : 'paradise', 'อัพเดท' : 'update', 'นาราย' : 'narai ', 'พรีเมี่ยม' : 'พรีเมียม',
    'บ๋วย' : 'บ๊วย', 'เฟรนไชส์' : 'แฟรนไชส์', 'บุพเฟ่' : 'บุฟเฟ่ต์', 'บุฟเฟ่' : 'บุฟเฟ่ต์', 'บุฟเฟ' : 'บุฟเฟ่ต์', 'บุฟเฟต' : 'บุฟเฟ่ต์', 'รสชาต' : 'รสชาติ', 'รสชาติิ' : 'รสชาติ'
}

# Cleansing
def cleanse_text(text):

    text = text.strip().lower()

    for old,new in word_correction_dict.items():
        text = text.replace(old, new)

    # Remove empty spaces
    text = re.sub(r'\u200b', '', text)
    text = re.sub(r'[\r\n]', '', text)

    # Replace punctuations
    text = re.sub(r'[ๆฯ!#$&%\"\'()*+,-./:;<=>?@\[\]\\^_`{}|~]',' ', text)

    # Replace numbers
    text = re.sub(r'\d',' ', text)
    text = re.sub(r'[0-9]','',text)
    text = re.sub(r'[๐-๙]','',text)

    return text

df['cleansed_text'] = df.apply(lambda x: cleanse_text(x['Headline'] + ' ' + x['Review']), axis=1)

# Sentence and Word Tokenization
df['tokens'] = df['cleansed_text'].apply(lambda x: [ w for st in sent_tokenize(x, engine='whitespace+newline') for w in sefr_cut.tokenize(st,k=100)])

stopwords = list(pythainlp.corpus.thai_stopwords())
removed = [' ', '  ', '\n', 'ร้าน', 'ที่' '(', ')', 'นะคะ', 'เค้า', 'ผม', 'คน']
screening = stopwords + removed

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening:
      merged = merged + ',' + word
  return merged[1:]

In [88]:
df['review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,cleansed_text,tokens,review_tokenized
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,"[[ที่, สำคัญ, ของ, ร้าน, นี้, คือ, บริการ, ดี,...","บริการ,ดีมาก,พนักงาน,น่ารัก,สะอาดสะอ้าน,ใส่ใจ,..."
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่...,"[[รสชาติ, เหมือน, ทุก, สาขา], [แต่, สาขา, นี้]...","นึกถึง,ชาบู,ญี่ปุ่น,คิดถึง,โม,โม่,พาราไดซ์,คุณ..."
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ชาบูพรีเมียมสไตล์ญี่ปุ่น เนื้อดีมากก มาทานช่วง...,"[[ชาบูพรีเมียม, สไตล์, ญี่ปุ่น], [เนื้อ, ดี, ม...","ทาน,นั่ง,โต๊ะ,หม้อ,โต๊ะ,ยังมี,ฉาก,กั้น,น้ำ,ซุป..."
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,เนื้อดี icecreamดี คุ้มค่าดี ถ้านึกถึงชาบูที่ม...,"[[เนื้อ, ดี], [icecream, ดี], [คุ้ม, ค่า, ดี],...","นึกถึง,ชาบู,เนื้อ,ราคา,โหด,เกินไป,นึกถึง,โม,โม..."
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...,"[[อาหาร, มี, คุณ, ภาพ], [บริการ, ดี, ค่ะ], [เด...","เดิน,หน้า,ได้กลิ่น,ชาบู,หอ,มมาก,โปร,บัตรเครดิต..."


In [89]:
documents = df['review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [90]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '           ', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '', '++', '2', '555', '600', ':', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ'

In [91]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

**Topic Modeling**

In [111]:
num_topics = 5
chunksize = 100   # size of the doc looked at every pass
passes = 20         # number of passes through documents
iterations = 100    
eval_every = 1     # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 325 ms, sys: 0 ns, total: 325 ms
Wall time: 326 ms


In [112]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.010400  0.061826       1        1  40.024643
0     -0.058132 -0.037709       2        1  24.941656
4      0.065398 -0.023939       3        1  20.419810
1      0.005108  0.002278       4        1   8.655333
3     -0.001975 -0.002455       5        1   5.958557, topic_info=        Term       Freq      Total Category  logprob  loglift
336        -  10.000000  10.000000  Default  30.0000  30.0000
26      ราคา  17.000000  17.000000  Default  29.0000  29.0000
20       บาท  13.000000  13.000000  Default  28.0000  28.0000
75      สาขา  12.000000  12.000000  Default  27.0000  27.0000
33     อาหาร  17.000000  17.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
194     หน้า   0.470476   5.962557   Topic5  -5.1857   0.2808
59   ญี่ปุ่น   0.420871   4.129512   Topic5  -5.2971   0.5368
8       คุ้ม   0.487020   9.818698   Topic5  -5.1512  -0.1834
151    เลือก   0.542612  16.767068   Topic5  -5.0431  -0.6104
272     กุ้ง   0.451333  11.289121   Topic5  -5.2273  -0.3990

[342 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
86        1  0.132114       
86        2  0.528457       
86        3  0.132114       
86        5  0.132114       
170       1  0.540553      (
...     ...       ...    ...
168       2  0.525613    ไม้
47        1  0.381155  ไอติม
47        2  0.381155  ไอติม
47        3  0.127052  ไอติม
48        1  0.909360      ​

[350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 5, 2, 4])

In [113]:
model.show_topic(1)

[('ราคา', 0.01864921),
 ('น้ำ', 0.018343037),
 ('บาท', 0.018200804),
 ('อาหาร', 0.016179185),
 ('ซุป', 0.01573729),
 ('ทาน', 0.015519349),
 ('-', 0.012817878),
 ('ดี', 0.012702778),
 ('คุ้ม', 0.011855222),
 ('รสชาติ', 0.011730485)]

In [114]:
df['topics'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [115]:
df[['Restaurant', 'cleansed_text', 'topics', 'score']]

,Restaurant,cleansed_text,topics,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,2,0.998795
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่...,3,0.997200
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ชาบูพรีเมียมสไตล์ญี่ปุ่น เนื้อดีมากก มาทานช่วง...,0,0.999274
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เนื้อดี icecreamดี คุ้มค่าดี ถ้านึกถึงชาบูที่ม...,2,0.997348
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...,2,0.997745
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,กินให้อิ่ม ที่ momo ร้านบุฟเฟ่ต์่ต์ ชาบูแนวญี่...,0,0.996097
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ชาบู สุกกี้สไลด์ญี่ปุ่นที่แท้ทรู number ...,2,0.998745
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูแฟรนไชส์ ที่ราคาเหมาะสมกับคุณภาพ ถือว่...,0,0.997743
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,อร่อยคุ้ม ของทานเล่นเยอะมาก มา านที่ขาบูชิต้อง...,1,0.996417
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ทานอีก ชอบกุ้งทอดเทมปุระ ใครชอบกุ้งทอดเทมปุระ ...,2,0.998358


In [119]:
pd.crosstab(df['Restaurant'], df['topics'])

topics,0,1,2,3,4
Restaurant,,,,,
Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,3,0,4,1,0
Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,0,1,5,0,2
ข้าน้อยขอชาบู,2,1,1,1,0


In [120]:
df.groupby('topics')['Rating'].describe()

,count,mean,std,min,25%,50%,75%,max
topics,,,,,,,,
0,5.0,4.0,0.707107,3.0,4.0,4.0,4.0,5.0
1,2.0,4.0,0.000000,4.0,4.0,4.0,4.0,4.0
2,10.0,4.0,0.666667,3.0,4.0,4.0,4.0,5.0
3,1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
4,2.0,3.0,0.000000,3.0,3.0,3.0,3.0,3.0
